In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score

import numpy as np

from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score

import numpy as np

from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

In [ ]:
# Load the datasets
train_data = pd.read_csv('train.csv')
val_data = pd.read_csv('val.csv')
test_data = pd.read_csv('test.csv')

lengthTestData = len(test_data)

In [ ]:
# Combine text data for preprocessing
text = pd.concat([train_data['text'], val_data['text']], ignore_index=True)
text_test = test_data['text']

In [ ]:
# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization and lowercase
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]  # Remove stopwords and lemmatize
    return ' '.join(filtered_tokens)

In [ ]:
text_preprocessed = text.apply(preprocess_text)

In [ ]:
print(text_preprocessed)

In [ ]:
print(text)

In [ ]:
# Initialize TF-IDF vectorizer without specifying max_features
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
# Fit and transform the preprocessed text data
tfidf_matrix = tfidf_vectorizer.fit_transform(text_preprocessed)

In [ ]:
# Get the number of unique tokens
num_unique_tokens = len(tfidf_vectorizer.get_feature_names_out())
print(num_unique_tokens)

In [ ]:
# Re-initialize TF-IDF vectorizer with the determined max_features
tfidf_vectorizer = TfidfVectorizer(max_features=num_unique_tokens)

In [ ]:
#Fit and transform the text data again with the updated max_features
tfidf_matrix = tfidf_vectorizer.fit_transform(text_preprocessed)

# Convert the TF-IDF matrix to a CSR (Compressed Sparse Row) matrix for efficient row-wise operations
csr_tfidf_matrix = csr_matrix(tfidf_matrix)

# Find the row index with the maximum number of filled values
max_features_row_index = csr_tfidf_matrix.getnnz(axis=1).argmax()

# Get the number of features in the document with the most filled values
max_features = csr_tfidf_matrix[max_features_row_index].count_nonzero()

svd = TruncatedSVD(n_components=int(max_features*0.1))
tfidf_matrix = svd.fit_transform(tfidf_matrix)

In [ ]:
#dense_tfidf_matrix = tfidf_matrix[:len(train_data)]
#dense_val_tfidf_matrix = tfidf_matrix[len(train_data):len(train_data) + len(val_data)]

# Merging the Validation and Training Data into one for a larger training dataset.
dense_tfidf_matrix = tfidf_matrix[:len(train_data) + len(val_data)]

In [ ]:
# Convert them into Arrays
train_labels = train_data['label'].values
val_labels = val_data['label'].values

dense_labels = np.concatenate((train_data['label'].values, val_data['label'].values), axis=0)

In [ ]:
# Create and train the SVM model
svm_model = SVC(kernel='rbf', C=5.0)
#svm_model = SVC(kernel='poly', C=20.0, degree=2, coef0=0.001)

#CROSS FOLD VALIDATION, DELETE IF NECESSARY

# Initialize KFold with the desired number of folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(svm_model, dense_tfidf_matrix, dense_labels, cv=kf, scoring='accuracy')

# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Average CV accuracy:", cv_scores.mean())

In [ ]:
svm_model.fit(dense_tfidf_matrix, dense_labels)

In [ ]:
text_test_preprocessed = text_test.apply(preprocess_text)
test_tfidf_matrix = tfidf_vectorizer.transform(text_test_preprocessed)
dense_test_tfidf_matrix = svd.transform(test_tfidf_matrix)
test_labels = test_data['label'].values

test_predictions = svm_model.predict(dense_test_tfidf_matrix)
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f'Test accuracy: {test_accuracy}')

In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(test_labels, test_predictions)
print(report)   

In [ ]:
#from joblib import dump
#dump(svm_model, 'svm_model_tfidf_9674.joblib')

In [ ]:
print(lengthTestData)

In [ ]:
test_accuracy*lengthTestData